# Drug Classification - Comparison of Machine Learning Models
Katlyn Goeujon-Mackness
06/05/2026

## Introduction
# Drug Classification - Comparison of Machine Learning Models

## Introduction

In this project, we explore different machine learning algorithms to predict the most suitable drug for a patient based on various health indicators. Using classification techniques, we analyze how factors such as **age, blood pressure, cholesterol levels, and sodium-to-potassium ratio** influence drug recommendations.

### Objectives:
- Implement and compare multiple **classification models**.
- Evaluate **performance metrics** to determine the most effective approach.
- Utilize **data visualization** to interpret patterns in drug prescriptions.

Through this analysis, we aim to optimize drug selection and provide **data-driven insights** into medication recommendations.

### Classification Algorithms
We will explore the following classification algorithms:
- Logistic Regression
- Decision Trees
- Random Forest Classifer
- K-Nearest Neighbors
- AdaBoost and Gradient Boost Classifiers

### Project Structure

1. Data Exploration
   - Loading the dataset
   - Understanding data structure
   - Handling missing values

2. Data Preprocessing
   - Encoding categorical variables
   - Feature scaling and transformation
   - Splitting dataset into train/test sets

3. Exploratory Data Analysis (EDA)
   - Visualizing distributions (histograms, box plots)
   - Correlation analysis
   - Insights from data

4. Model Selection & Training
   - Choosing classification models (e.g., Logistic Regression, Decision Trees, Random Forests)
   - Implementing machine learning algorithms
   - Performance evaluation metrics

5. Model Comparison
   - Comparing accuracy, precision, recall, and F1-score
   - Hyperparameter tuning (Grid Search, Random Search)
   - Feature importance analysis

6. Conclusion
   - Best-performing model summary
   - Potential improvements
   - Future applications of drug classification

## Data Exploration
- Loading the dataset
- Understanding data structure
- Handling missing values

## Data Preprocessing
   - Encoding categorical variables
   - Feature scaling and transformation
   - Splitting dataset into train/test sets

## Exploratory Data Analysis (EDA)
- Visualizing distributions (histograms, box plots)
- Correlation analysis
- Insights from data

## Model Selection & Training
- Choosing classification models (e.g., Logistic Regression, Decision Trees, Random Forests)
- Implementing machine learning algorithms
- Performance evaluation metrics

## Model Comparison
- Comparing accuracy, precision, recall, and F1-score
- Hyperparameter tuning (Grid Search, Random Search)
- Feature importance analysis

## Conclusion
- Best-performing model summary
- Potential improvements
- Future applications of drug classification